# Statistiques descriptives à partir des données importées

In [8]:
%pip install tabula
import pandas as pd
import tabula
import numpy as np
from IPython.utils import io
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

Note: you may need to restart the kernel to use updated packages.


In [9]:
try:
    with io.capture_output():
        %run -i "Calendrier.ipynb"
except Exception as e:
    display(e)

In [10]:
try:
    with io.capture_output():
        %run -i "Informations lieux des compétitions.ipynb"
except Exception as e:
    display(e)

In [41]:
try:
    with io.capture_output():
        %run -i "Informations lignes métro.ipynb"
except Exception as e:
    display(e)

# Statistiques à partir du notebook Informations lieux des compétitions

In [12]:
# Suppression des tirets et mise en majuscule du début de chaque mot dans la colonne "Lieu"
formatted_lieux = df['Lieu'].apply(lambda x: x.replace('-', ' ').title())

# Création du diagramme à barres interactif avec Plotly en utilisant les nouvelles étiquettes
fig = go.Figure(data=[
    go.Bar(x=formatted_lieux, y=df["Nombre de personnes à l'événément"])
])

fig.update_layout(
    title="Nombre de personnes attendues aux lieux de compétitions",
    xaxis=dict(title='Lieu'),
    yaxis=dict(title="Nombre de personnes"),
    xaxis_tickangle=-45  # Rotation des étiquettes x pour une meilleure lisibilité
)

fig.show()


## Statistiques à partir du notebook informations lignes métro

In [13]:

# Pour chaque tableau dans le dictionnaire
for k in numéro:
    k_str = str(k)
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Assurez-vous que la colonne 'Part de la station dans le trafic annuel de la ligne' est en pourcentage
    current_table['Part du trafic de la station par rapport à toute la ligne'] *= 100

    # Créez le graphique en camembert pour le tableau actuel
    fig = px.pie(
        current_table,
        values='Part du trafic de la station par rapport à toute la ligne',
        names='Station',
        title=f"Répartition du trafic pour la ligne numéro {k_str}"
    )
    fig.show()


# Statistiques descriptives à partir de notre programme 

In [37]:
try:
    with io.capture_output():
        %run -i "Programmes.ipynb"
except Exception as e:
    display(e)

In [42]:
dates = pd.date_range(start='2024-07-24', end='2024-08-11', freq='D').strftime('%Y-%m-%d')

# Liste des heures de 00:00 à 23:00
heures = [f"{hour:02d}:00" for hour in range(24)]
display(heures)

resultats_total_station = []

for date in dates:
    for heure in heures:
        try:
            total_station, _ = info_trafic(date, heure, station_info, ligne_info)

            if total_station is not None:
                resultats_total_station.append((date, heure, total_station))

        except ValueError as e:
            print(e)
            print("Erreur dans le calcul des affluences.")

# Convertir la liste en DataFrame
df_resultats_station = pd.DataFrame(resultats_total_station, columns=['Date', 'Heure', 'Total Station'])
df_resultats_station.set_index(['Date', 'Heure'], inplace=True)

print(df_resultats_station)


['00:00',
 '01:00',
 '02:00',
 '03:00',
 '04:00',
 '05:00',
 '06:00',
 '07:00',
 '08:00',
 '09:00',
 '10:00',
 '11:00',
 '12:00',
 '13:00',
 '14:00',
 '15:00',
 '16:00',
 '17:00',
 '18:00',
 '19:00',
 '20:00',
 '21:00',
 '22:00',
 '23:00']

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyError: 'Part de la station dans le trafic annuel de la ligne'